<a href="https://colab.research.google.com/github/tolgagonen/google-collab-10.03/blob/main/pozitif_negatif_twitter_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [15]:
df = pd.read_csv("newcsv.csv",encoding='latin1')
#utf-8 codec can t decode hatası çözümü için latin1 kullandım , bunla birlikte 
#türkçede ki özel karakterleri okuyabiliyor

In [16]:
df.shape

(11119, 2)

In [17]:
df.head()

,type,text
0,Negatif,doða aðzýmýza sýçsa hakký var
1,Pozitif,anne bir sanatçýdýr güzel eseri yavrusudur
2,Negatif,ibrahimin oðlunu koruyan tanrý ben sokayým
3,Negatif,köpeðim suratýna sýçsýn senin namussuz karý
4,Negatif,ben söðüþledim birazda sen söðüþle


In [18]:
print((df.type == "Negatif").sum()) # negatif cümle sayısı
print((df.type == "Pozitif").sum()) # pozitif cümle sayısı

5004
6115


In [23]:
# Tekrar işleme , burada remove_ULR de linkleri yok ediyoruz
#altındaki punct kısmında ise noktaları
import re
import string

def remove_URL(text):
    url =re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", str(text))

def remove_punct(text):
    translator = str.maketrans("","", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
df["text"] = df.text.map(remove_URL)
df["text"] = df.text.map(remove_punct)

In [25]:
#Twitter gibi uygulamalarda genellikle noktalama işaretlerine çok dikkat edilmez
#Bu yüzden cümlelerin sonunda stopwords dediğimiz cümleleri bitirme kelimeleri bulunur
#Bunlardan burada kurtulmak için hazırlanıyoruz.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#stop words A stop words is a commonly used word (such as "the" , "a")

stop= set(stopwords.words("english"))
stop2= set(stopwords.words("turkish"))

def remove_stopwords(text):
  filtered_words = [word.lower() for word in text.split() if word.lower() not in stop2]
  return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
df["text"] = df.text.map(remove_stopwords)

In [27]:
df.text

0                      doða aðzýmýza sýçsa hakký var
1         anne bir sanatçýdýr güzel eseri yavrusudur
2         ibrahimin oðlunu koruyan tanrý ben sokayým
3        köpeðim suratýna sýçsýn senin namussuz karý
4                 ben söðüþledim birazda sen söðüþle
                            ...                     
11114                               bugün hava güzel
11115                        allah akýl fikir versin
11116                            allah belaný versin
11117                   allah akýl fikir versin sana
11118                        allah akýl fikir versin
Name: text, Length: 11119, dtype: object

In [28]:
#Burada tüm farklı kelimeleri sayıyoruz
from collections import Counter

def counter_word(text_col):
  count = Counter()
  for text in text_col.values:
    for word in text.split():
      count[word] += 1
  return count

counter = counter_word(df.text)

In [29]:
len(counter)

36005

In [31]:
counter.most_common(5) 
# En sık kullanılan 5 kelimeye erişiyoruz.

[('bir', 2031), ('ben', 585), ('kadar', 570), ('var', 540), ('sen', 491)]

In [32]:
num_unique_words = len(counter) # Eşsiz Kelime Sayısı

In [70]:
#Burada type tarafındaki negatif pozitif datasını 0 ve 1 olarak değiştiriyorum
df['type'] = (df['type'] == "Pozitif").astype(int)



In [72]:
df.head()

,type,text
0,0,doða aðzýmýza sýçsa hakký var
1,1,anne bir sanatçýdýr güzel eseri yavrusudur
2,0,ibrahimin oðlunu koruyan tanrý ben sokayým
3,0,köpeðim suratýna sýçsýn senin namussuz karý
4,0,ben söðüþledim birazda sen söðüþle


In [89]:
#Eğitim için elimizdeki datanın yüzde 85'ini kullanıcam
#Geri kalan ise valdiation için kullanıcam
train_size = int(df.shape[0] * 0.85)
train_df = df[:train_size]
val_df = df[train_size:]

#Textleri ve type 'ları ayırıyorum
train_sentences = train_df.text.to_numpy()
train_labels = train_df.type.to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df.type.to_numpy()


In [90]:
train_sentences.shape , val_sentences.shape

((9451,), (1668,))

In [91]:
#tokenizer paragrafları ve cümleleri daha kolay anlam atanabilecek daha küçük birimlere ayırır

from tensorflow.keras.preprocessing.text import Tokenizer

#vectorize a text corpus by turning each text into a sequance of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences)

In [92]:
#Her kelimenin kendine özel bir indexi var
word_index = tokenizer.word_index

In [93]:
word_index

{'bir': 1,
 'ben': 2,
 'kadar': 3,
 'var': 4,
 'sen': 5,
 'bi': 6,
 'senin': 7,
 'seni': 8,
 'yok': 9,
 'orospu': 10,
 'mi': 11,
 'olsun': 12,
 'deðil': 13,
 'amk': 14,
 'lan': 15,
 'güzel': 16,
 'böyle': 17,
 'sonra': 18,
 'allah': 19,
 'sana': 20,
 'iyi': 21,
 'benim': 22,
 'artýk': 23,
 'bak': 24,
 'olan': 25,
 'gün': 26,
 'bile': 27,
 'çocuðu': 28,
 'pezevenk': 29,
 'aptal': 30,
 'beni': 31,
 'adam': 32,
 'bana': 33,
 'büyük': 34,
 'insan': 35,
 'önce': 36,
 'aq': 37,
 'zaman': 38,
 'tek': 39,
 'olarak': 40,
 'yine': 41,
 'köpek': 42,
 'þimdi': 43,
 'sadece': 44,
 'göt': 45,
 'olur': 46,
 'bunu': 47,
 'öyle': 48,
 'sizin': 49,
 'türkiye': 50,
 'zaten': 51,
 'türk': 52,
 'git': 53,
 'kendi': 54,
 'çocuk': 55,
 'tam': 56,
 'ilk': 57,
 'can': 58,
 'iki': 59,
 'kýz': 60,
 'veren': 61,
 'sokim': 62,
 'bugün': 63,
 'kadýn': 64,
 'oldu': 65,
 'götlek': 66,
 'bizim': 67,
 'ayný': 68,
 'olmak': 69,
 'herkes': 70,
 'þerefsiz': 71,
 'versin': 72,
 'pislik': 73,
 'ulan': 74,
 'ayasofya': 75,
 

In [94]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [95]:
print(train_sentences[10:15])
print(train_sequences[10:15])

['chp genel baþkaný kemal kýlýçdaroðlu nun eþi selvi kýlýçdaroðlu hazine maliye bakaný berat albayrak ýn eþi esra albayrak telefonla arayarak yeni doðan çocuðu ilgili sosyal medyadan yapýlan hakaret içerikli paylaþýmlar nedeniyle duyduðu üzüntüyü paylaþtý'
 'beyin göçünü engellemeye çalýþtýðýmýz gençlere sosyal medyayý yasaklamak gerçekten dahiyane bir fikir'
 'doða ana kullanýlmayan depoyu devralýyor'
 'tarlasýný çekerken beni çekmedin söylenenen dayýnýn kendinin çekildiðinden habersiz oluþu'
 'babanesini seven yoktur heralde anneneler kadar minnoþsa baanneler kadar çekilmez oluyo inþlah babane olmam ilerde aq kimse sevmez beni']
[[462, 333, 317, 463, 2402, 1090, 1747, 5580, 2402, 5581, 2965, 806, 5582, 2966, 94, 1747, 2008, 2966, 5583, 9813, 88, 5584, 28, 390, 173, 2967, 408, 356, 2968, 5585, 979, 9814, 9815, 5586], [636, 5587, 5588, 5589, 1213, 173, 5590, 3841, 106, 5591, 1, 434], [1086, 514, 5592, 9816, 9817], [9818, 9819, 31, 9820, 9821, 9822, 9823, 9824, 3842, 9825], [9826, 1214,

In [96]:
# dizileri aynı uzunluğa sahip olacak şekilde doldurun
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length=20
 
train_padded= pad_sequences(train_sequences , maxlen=max_length , padding="post",truncating="post")
val_padded= pad_sequences(val_sequences, maxlen=max_length, padding="post" , truncating="post")
train_padded.shape , val_padded.shape

((9451, 20), (1668, 20))

In [97]:
train_padded[10]

array([ 462,  333,  317,  463, 2402, 1090, 1747, 5580, 2402, 5581, 2965,
        806, 5582, 2966,   94, 1747, 2008, 2966, 5583, 9813], dtype=int32)

In [98]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

chp genel baþkaný kemal kýlýçdaroðlu nun eþi selvi kýlýçdaroðlu hazine maliye bakaný berat albayrak ýn eþi esra albayrak telefonla arayarak yeni doðan çocuðu ilgili sosyal medyadan yapýlan hakaret içerikli paylaþýmlar nedeniyle duyduðu üzüntüyü paylaþtý
[462, 333, 317, 463, 2402, 1090, 1747, 5580, 2402, 5581, 2965, 806, 5582, 2966, 94, 1747, 2008, 2966, 5583, 9813, 88, 5584, 28, 390, 173, 2967, 408, 356, 2968, 5585, 979, 9814, 9815, 5586]
[ 462  333  317  463 2402 1090 1747 5580 2402 5581 2965  806 5582 2966
   94 1747 2008 2966 5583 9813]


In [99]:
# Burada indexleri tersine çevirerek kontrol ediyoruz
reverse_word_index= dict([(idx , word) for  (word,idx) in word_index.items()])

In [100]:
def decode(sequence):
  return " ".join([reverse_word_index.get(idx , "?") for idx in sequence])

In [101]:
decoded_text = decode(train_sequences[10])
print(train_sequences[10])
print(decoded_text)

[462, 333, 317, 463, 2402, 1090, 1747, 5580, 2402, 5581, 2965, 806, 5582, 2966, 94, 1747, 2008, 2966, 5583, 9813, 88, 5584, 28, 390, 173, 2967, 408, 356, 2968, 5585, 979, 9814, 9815, 5586]
chp genel baþkaný kemal kýlýçdaroðlu nun eþi selvi kýlýçdaroðlu hazine maliye bakaný berat albayrak ýn eþi esra albayrak telefonla arayarak yeni doðan çocuðu ilgili sosyal medyadan yapýlan hakaret içerikli paylaþýmlar nedeniyle duyduðu üzüntüyü paylaþtý


In [102]:
#LSTM modeli yapıyoruz.

from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32 , input_length=max_length))


model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 32)            1152160   
                                                                 
 lstm_4 (LSTM)               (None, 64)                24832     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,177,057
Trainable params: 1,177,057
Non-trainable params: 0
_________________________________________________________________


In [103]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim= keras.optimizers.Adam(lr=0.001)
metrics= ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [104]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)
#training kısmını burada yaptık doğruluk %71 -datanın %75 ile eğitim yaptığımda
#training kısmını burada yaptık doğruluk %69 -datanın %85 ile eğitim yaptığımda

Epoch 1/20
296/296 - 12s - loss: 0.4894 - accuracy: 0.7453 - val_loss: 0.6319 - val_accuracy: 0.7212 - 12s/epoch - 42ms/step
Epoch 2/20
296/296 - 8s - loss: 0.1348 - accuracy: 0.9521 - val_loss: 0.7646 - val_accuracy: 0.6954 - 8s/epoch - 26ms/step
Epoch 3/20
296/296 - 9s - loss: 0.0377 - accuracy: 0.9891 - val_loss: 0.9550 - val_accuracy: 0.7032 - 9s/epoch - 29ms/step
Epoch 4/20
296/296 - 9s - loss: 0.0219 - accuracy: 0.9941 - val_loss: 1.4337 - val_accuracy: 0.6841 - 9s/epoch - 29ms/step
Epoch 5/20
296/296 - 8s - loss: 0.0146 - accuracy: 0.9966 - val_loss: 0.9831 - val_accuracy: 0.7044 - 8s/epoch - 27ms/step
Epoch 6/20
296/296 - 9s - loss: 0.0083 - accuracy: 0.9975 - val_loss: 1.5046 - val_accuracy: 0.7032 - 9s/epoch - 30ms/step
Epoch 7/20
296/296 - 9s - loss: 0.0073 - accuracy: 0.9980 - val_loss: 1.8129 - val_accuracy: 0.6990 - 9s/epoch - 30ms/step
Epoch 8/20
296/296 - 8s - loss: 0.0088 - accuracy: 0.9982 - val_loss: 1.5864 - val_accuracy: 0.7050 - 8s/epoch - 27ms/step
Epoch 9/20
296

In [105]:
predictions = model.predict(train_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

296/296 [==============================] - 3s 7ms/step


In [106]:
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

['chp genel baþkaný kemal kýlýçdaroðlu nun eþi selvi kýlýçdaroðlu hazine maliye bakaný berat albayrak ýn eþi esra albayrak telefonla arayarak yeni doðan çocuðu ilgili sosyal medyadan yapýlan hakaret içerikli paylaþýmlar nedeniyle duyduðu üzüntüyü paylaþtý'
 'beyin göçünü engellemeye çalýþtýðýmýz gençlere sosyal medyayý yasaklamak gerçekten dahiyane bir fikir'
 'doða ana kullanýlmayan depoyu devralýyor'
 'tarlasýný çekerken beni çekmedin söylenenen dayýnýn kendinin çekildiðinden habersiz oluþu'
 'babanesini seven yoktur heralde anneneler kadar minnoþsa baanneler kadar çekilmez oluyo inþlah babane olmam ilerde aq kimse sevmez beni'
 'merhaba konuyla ilgili yardýmcý olmak isteriz rtibat numaranýzý direkt mesaj olarak bize iletir misiniz'
 'nietzsche böyle buyurdu zerdüþt te önce sen kendini inþa etmelisin dimdik bir beden dimdik bir ruhla diyor ekliyor derisini deðiþtirmeyen yýlan ölmeye mahkumdur durum fikirlerini deðiþtirmeyen zihinler geçerlidir sabit fikir büyük hapishanedir'
 'hiçbir